In [1]:
import os
import json
import glob 

import numpy as np
import pandas as pd

import cv2

In [2]:
datasets_dir = os.path.abspath("../data/tator/grid/datasets")
datasets_dir

datasets_folders = glob.glob(os.path.join(datasets_dir, "*"))
datasets_folders

['c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\dreiss-annotations-wscnms-dropcam',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\dreiss-annotations-wscnms-usgs-auv',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\hard-mud-annotations-wscnms-dropcam',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\hard-mud-annotations-wscnms-usgs-auv',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\sand-annotations-wscnms-dropcam',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\sand-annotations-wscnms-usgs-auv',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\sav-annotations-wscnms-dropcam',
 'c:\\Users\\jordan\\Documents\\GitHub\\CoralNet-Toolbox\\data\\tator\\grid\\datasets\\sav-annotations-wscnms-usgs-auv',
 'c:\\Users\\jorda

In [ ]:
cell_data = []

for dataset_folder in datasets_folders:
    dataset = dataset_folder
    images = glob.glob(os.path.join(dataset, "*.png"))
    json_files = glob.glob(os.path.join(dataset, "*.json"))

    for json_file in json_files:
        with open(json_file, 'r') as f:
            d = json.load(f)
            json_data = [d]
            
        image_name = os.path.basename(json_file).split("_version")[0] + ".png"
            
        for j in json_data:
            keys = j['grid'].keys()
            for k in keys:
                d = j['grid'][k]
                for sub_key in d.keys():
                    cell = d[sub_key]
                    if not len(cell['labels']):
                        continue
                    
                    label = cell['labels'][0]
                    top_left_x = cell['top_left_x']
                    top_left_y = cell['top_left_y']
                    width = cell['width']
                    height = cell['height']
                    
                    col = cell['center_x']
                    row = cell['center_y']
                    patch_size = min(width, height)
                    # Split off the "_version*.json" part of the filename
                    image_name = os.path.basename(json_file).split("_version")[0] + ".png"
                    cell_data.append([image_name, label, col, row, patch_size])
                
    df = pd.DataFrame(cell_data, columns=["Name", "Label", "Column", "Row", "Patch_Size"])
    df = df.drop_duplicates()
    df.to_csv(f"{datasets_dir}/grid_annotations.csv", index=False)